Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle


First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
batch_size = 128
image_size = 28
num_labels = 10
wd_coeff = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  classification_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  weightdecay_loss = wd_coeff * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  loss = classification_loss + weightdecay_loss

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 22.719421
Minibatch accuracy: 6.2%
Validation accuracy: 8.7%
Minibatch loss at step 500: 2.860222
Minibatch accuracy: 77.3%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 1.635398
Minibatch accuracy: 75.8%
Validation accuracy: 78.8%
Minibatch loss at step 1500: 1.400837
Minibatch accuracy: 75.8%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 1.207291
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 2500: 0.974452
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.702569
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Test accuracy: 88.7%


In [8]:
#SGD with relu
batch_size = 128
relu_count = 1024 #hidden nodes count
wd_coeff = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

    # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, relu_count]))
  biases_1 = tf.Variable(tf.zeros([relu_count]))
  # send relu to final nn layer
  weights_2 = tf.Variable(tf.truncated_normal([relu_count, num_labels]))

  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation. (#layer_1 -> layer_2(relu) -> layer_3)
  logits = tf.matmul( tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2

  classification_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  regularizers = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) +
                  tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2))
 
  weightdecay_loss = wd_coeff * regularizers
  loss = classification_loss + weightdecay_loss
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)


In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 594.930847
Minibatch accuracy: 18.0%
Validation accuracy: 26.3%
Minibatch loss at step 500: 196.308350
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 115.435341
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 71.065117
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 2000: 42.478470
Minibatch accuracy: 80.5%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 25.637028
Minibatch accuracy: 82.0%
Validation accuracy: 85.6%
Minibatch loss at step 3000: 15.578238
Minibatch accuracy: 85.2%
Validation accuracy: 86.4%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [5]:
### Define your architecture here.

# Hyper parameters
batch_size = 128
image_size = 28
input = image_size*image_size
f1_out = 1024
f2_out = 512
num_labels = 10
wd_coeff = 0.0
drop_out1 = 0.8
drop_out2 = 0.5
drop_out3 = 0.5
initial_learning_rate = 1e-4


# Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, input))
tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

init = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
weights = {
           'f1': tf.get_variable("f1", shape=[input, f1_out], initializer=init),
           'f2': tf.get_variable("f2", shape=[f1_out, f2_out], initializer=init),
           'out': tf.get_variable("out", shape=[f2_out, num_labels], initializer=init)}

bias = {
          'f1': tf.Variable(tf.zeros([f1_out])),
          'f2': tf.Variable(tf.zeros([f2_out])),
          'out': tf.Variable(tf.zeros([num_labels]))}

regularizers = (tf.nn.l2_loss(weights['f1']) + tf.nn.l2_loss(bias['f1']) +
                  tf.nn.l2_loss(weights['f2']) + tf.nn.l2_loss(bias['f2']) +
                  tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(bias['out']))

keep_prob1 = tf.placeholder(tf.float32)
keep_prob2 = tf.placeholder(tf.float32)
keep_prob3 = tf.placeholder(tf.float32)


In [6]:
def Model(x, use_dropout=False):    
                                          
    # Layer 1: Fully Connected. 
    if use_dropout:
        reshape = tf.nn.dropout(x, keep_prob1)    
    h_layer1 = tf.matmul(x, weights['f1']) + bias['f1']
    h_layer1 = tf.nn.relu(h_layer1)
    #h_layer1 = tf.nn.batch_normalization(h_layer1, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)
   
    # Layer 2: Fully Connected. 
    if use_dropout:
        h_layer1 = tf.nn.dropout(h_layer1, keep_prob2)
    h_layer2 = tf.matmul(h_layer1, weights['f2']) + bias['f2']
    h_layer2 = tf.nn.relu(h_layer2)
    #h_layer2 = tf.nn.batch_normalization(h_layer2, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)
    
    # Layer 3: Fully Connected. 
    if use_dropout:
        h_layer2 = tf.nn.dropout(h_layer2, keep_prob3)
    logits = tf.matmul(h_layer2, weights['out']) + bias['out']

    return logits


In [7]:
# Training computation. (#layer_1 -> layer_2(relu) -> layer_3)
logits = Model(tf_train_dataset, True)

classification_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

weightdecay_loss = wd_coeff * regularizers
loss = classification_loss + weightdecay_loss
    
# Optimizer.
global_step = tf.Variable(0)
lr = tf.train.exponential_decay(initial_learning_rate, global_step, 3500, 0.98, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
#training_operation = optimizer.minimize(loss)
training_operation = optimizer.minimize(loss, global_step=global_step)
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits)

valid_logits = Model(tf_valid_dataset)
valid_prediction = tf.nn.softmax(valid_logits)

test_logits = Model(tf_test_dataset)
test_prediction = tf.nn.softmax(test_logits)


In [8]:
EPOCHS = 8
num_steps = EPOCHS * 3500
from sklearn.utils import shuffle

with tf.Session() as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    if (step % 3500 == 0):
        train_dataset, train_labels = shuffle(train_dataset, train_labels)
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob1: drop_out1, keep_prob2: drop_out2, keep_prob3: drop_out3}

    _, l, predictions = session.run([training_operation, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.443675
Minibatch accuracy: 8.6%
Validation accuracy: 12.3%
Minibatch loss at step 2000: 0.501768
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Minibatch loss at step 4000: 0.432948
Minibatch accuracy: 86.7%
Validation accuracy: 86.9%
Minibatch loss at step 6000: 0.373425
Minibatch accuracy: 89.1%
Validation accuracy: 87.9%
Minibatch loss at step 8000: 0.431385
Minibatch accuracy: 89.8%
Validation accuracy: 88.6%
Minibatch loss at step 10000: 0.377390
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 12000: 0.426697
Minibatch accuracy: 86.7%
Validation accuracy: 89.5%
Minibatch loss at step 14000: 0.299272
Minibatch accuracy: 89.8%
Validation accuracy: 89.6%
Minibatch loss at step 16000: 0.420565
Minibatch accuracy: 87.5%
Validation accuracy: 90.2%
Minibatch loss at step 18000: 0.398190
Minibatch accuracy: 89.1%
Validation accuracy: 90.2%
Minibatch loss at step 20000: 0.288998
Minibatch accuracy: 89.1%
Validation a